In [98]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from torchsummary import summary
import numpy as np

In [99]:
# 데이터셋 인스턴스 생성

df = pd.read_csv(r"C:\ai_class\week7\BP_data.csv")  # CSV 파일 경로
df

,Patient_Number,Blood_Pressure_Abnormality,Level_of_Hemoglobin,Genetic_Pedigree_Coefficient,Age,BMI,Sex,Pregnancy,Smoking,Physical_activity,salt_content_in_the_diet,alcohol_consumption_per_day,Level_of_Stress,Chronic_kidney_disease,Adrenal_and_thyroid_disorders
0,1,1,11.28,0.90,34,23,1,1.0,0,45961,48071,NaN,2,1,1
1,2,0,9.75,0.23,54,33,1,NaN,0,26106,25333,205.0,3,0,0
2,3,1,10.79,0.91,70,49,0,NaN,0,9995,29465,67.0,2,1,0
3,4,0,11.00,0.43,71,50,0,NaN,0,10635,7439,242.0,1,1,0
4,5,1,14.17,0.83,52,19,0,NaN,0,15619,49644,397.0,2,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,1,10.14,0.02,69,26,1,NaN,1,26118,47568,144.0,3,1,0
1996,1997,1,11.77,1.00,24,45,1,1.0,1,2572,8063,NaN,3,1,1
1997,1998,1,16.91,0.22,18,42,0,NaN,0,14933,24753,NaN,2,1,1
1998,1999,0,11.15,0.72,46,45,1,NaN,1,18157,15275,253.0,3,0,1


In [100]:
df.columns

Index(['Patient_Number', 'Blood_Pressure_Abnormality', 'Level_of_Hemoglobin',
       'Genetic_Pedigree_Coefficient', 'Age', 'BMI', 'Sex', 'Pregnancy',
       'Smoking', 'Physical_activity', 'salt_content_in_the_diet',
       'alcohol_consumption_per_day', 'Level_of_Stress',
       'Chronic_kidney_disease', 'Adrenal_and_thyroid_disorders'],
      dtype='object')

In [101]:
df.isnull().sum()

Patient_Number                      0
Blood_Pressure_Abnormality          0
Level_of_Hemoglobin                 0
Genetic_Pedigree_Coefficient       92
Age                                 0
BMI                                 0
Sex                                 0
Pregnancy                        1558
Smoking                             0
Physical_activity                   0
salt_content_in_the_diet            0
alcohol_consumption_per_day       242
Level_of_Stress                     0
Chronic_kidney_disease              0
Adrenal_and_thyroid_disorders       0
dtype: int64

In [102]:
columns_to_delete = ['Pregnancy']
df = df.drop(columns=columns_to_delete)
df = df.dropna()
df.isnull().sum()

Patient_Number                   0
Blood_Pressure_Abnormality       0
Level_of_Hemoglobin              0
Genetic_Pedigree_Coefficient     0
Age                              0
BMI                              0
Sex                              0
Smoking                          0
Physical_activity                0
salt_content_in_the_diet         0
alcohol_consumption_per_day      0
Level_of_Stress                  0
Chronic_kidney_disease           0
Adrenal_and_thyroid_disorders    0
dtype: int64

In [103]:
# 데이터와 타겟 분리
X = df.drop('Blood_Pressure_Abnormality', axis=1)
y = df['Blood_Pressure_Abnormality']

In [104]:
X.head()

,Patient_Number,Level_of_Hemoglobin,Genetic_Pedigree_Coefficient,Age,BMI,Sex,Smoking,Physical_activity,salt_content_in_the_diet,alcohol_consumption_per_day,Level_of_Stress,Chronic_kidney_disease,Adrenal_and_thyroid_disorders
1,2,9.75,0.23,54,33,1,0,26106,25333,205.0,3,0,0
2,3,10.79,0.91,70,49,0,0,9995,29465,67.0,2,1,0
3,4,11.00,0.43,71,50,0,0,10635,7439,242.0,1,1,0
4,5,14.17,0.83,52,19,0,0,15619,49644,397.0,2,0,0
6,7,11.69,0.75,43,41,1,0,38369,32967,206.0,3,1,1


In [105]:
Y = pd.get_dummies(y).values
Y

array([[ True, False],
       [False,  True],
       [ True, False],
       ...,
       [False,  True],
       [ True, False],
       [False,  True]])

In [106]:
X = X.values
X

array([[2.000e+00, 9.750e+00, 2.300e-01, ..., 3.000e+00, 0.000e+00,
        0.000e+00],
       [3.000e+00, 1.079e+01, 9.100e-01, ..., 2.000e+00, 1.000e+00,
        0.000e+00],
       [4.000e+00, 1.100e+01, 4.300e-01, ..., 1.000e+00, 1.000e+00,
        0.000e+00],
       ...,
       [1.996e+03, 1.014e+01, 2.000e-02, ..., 3.000e+00, 1.000e+00,
        0.000e+00],
       [1.999e+03, 1.115e+01, 7.200e-01, ..., 3.000e+00, 0.000e+00,
        1.000e+00],
       [2.000e+03, 1.136e+01, 9.000e-02, ..., 1.000e+00, 1.000e+00,
        0.000e+00]])

In [107]:
# Standardize the data
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Split the dataset into training and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(X, Y, test_size=0.15, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.15/0.85, random_state=42)

In [108]:
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

((1173, 13), (252, 13), (252, 13), (1173, 2), (252, 2), (252, 2))

In [109]:
# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

In [110]:
# Create DataLoader
train_dataset = TensorDataset(X_train, y_train)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = TensorDataset(X_test, y_test)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

val_dataset = TensorDataset(X_test, y_test)
val_dataloader  = DataLoader(val_dataset, batch_size=32, shuffle=False)

In [111]:
X_train.shape, X_test.shape, X_val.shape, y_train.shape, y_test.shape, y_val.shape

(torch.Size([1173, 13]),
 torch.Size([252, 13]),
 torch.Size([252, 13]),
 torch.Size([1173, 2]),
 torch.Size([252, 2]),
 torch.Size([252, 2]))

In [112]:
class BPDense(nn.Module):
    def __init__(self):
        super(BPDense, self).__init__()
        self.fc1 = nn.Linear(13, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 64)
        self.fc4 = nn.Linear(64, 32)
        self.fc5 = nn.Linear(32, 2)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.relu(self.fc3(x))
        x = torch.relu(self.fc4(x))
        x = self.fc5(x)
        return x

# Initialize the model, loss function, and optimizer
model = BPDense()

In [113]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [114]:
# Variables to store loss and accuracy
train_losses = []
val_accuracies = []

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_dataloader:
        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average loss over an epoch
    train_losses.append(running_loss / len(train_dataloader))

    # Evaluate on test data
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in val_dataloader:
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            # labels가 원-핫 인코딩되어 [32, 2]의 shape라면, argmax로 [32]의 클래스 인덱스로 변환
            labels_idx = labels.argmax(dim=1)
            total += labels.size(0)
            correct += (predicted == labels_idx).sum().item()

    accuracy = 100 * correct / total
    val_accuracies.append(accuracy)

    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {train_losses[-1]:.4f}, Accuracy: {accuracy:.2f}%")

print("Training complete.")

Epoch 1/20, Loss: 0.6865, Accuracy: 67.06%
Epoch 2/20, Loss: 0.5759, Accuracy: 67.86%
Epoch 3/20, Loss: 0.5020, Accuracy: 70.63%
Epoch 4/20, Loss: 0.4515, Accuracy: 74.21%
Epoch 5/20, Loss: 0.4095, Accuracy: 76.59%
Epoch 6/20, Loss: 0.3678, Accuracy: 79.37%
Epoch 7/20, Loss: 0.3288, Accuracy: 80.16%
Epoch 8/20, Loss: 0.2973, Accuracy: 81.75%
Epoch 9/20, Loss: 0.2714, Accuracy: 82.54%
Epoch 10/20, Loss: 0.2500, Accuracy: 83.33%
Epoch 11/20, Loss: 0.2275, Accuracy: 84.13%
Epoch 12/20, Loss: 0.2154, Accuracy: 82.94%
Epoch 13/20, Loss: 0.2009, Accuracy: 81.75%
Epoch 14/20, Loss: 0.1949, Accuracy: 83.33%
Epoch 15/20, Loss: 0.1781, Accuracy: 83.73%
Epoch 16/20, Loss: 0.1599, Accuracy: 83.33%
Epoch 17/20, Loss: 0.1564, Accuracy: 85.32%
Epoch 18/20, Loss: 0.1471, Accuracy: 83.73%
Epoch 19/20, Loss: 0.1358, Accuracy: 84.52%
Epoch 20/20, Loss: 0.1273, Accuracy: 81.75%
Training complete.


In [115]:
# Evaluation
model.eval()
all_labels = []
all_predictions = []
with torch.no_grad():
    for inputs, labels in test_dataloader:
        # 만약 labels가 원-핫 인코딩 형태라면, 클래스 인덱스로 변환 (예: [batch_size, num_classes] -> [batch_size])
        if labels.dim() > 1 and labels.shape[1] > 1:
            labels = labels.argmax(dim=1)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        all_labels.extend(labels.cpu().numpy())
        all_predictions.extend(predicted.cpu().numpy())

# Convert to numpy arrays
all_labels = np.array(all_labels)
all_predictions = np.array(all_predictions)

# Calculate metrics
conf_matrix = confusion_matrix(all_labels, all_predictions)
f1 = f1_score(all_labels, all_predictions, average='weighted')
precision = precision_score(all_labels, all_predictions, average='weighted')
recall = recall_score(all_labels, all_predictions, average='weighted')

# Calculate specificity for each class
specificity = []
for i in range(conf_matrix.shape[0]):
    tn = conf_matrix.sum() - (conf_matrix[i, :].sum() + conf_matrix[:, i].sum() - conf_matrix[i, i])
    fp = conf_matrix[:, i].sum() - conf_matrix[i, i]
    specificity.append(tn / (tn + fp))

print(f'Confusion Matrix:\n{conf_matrix}')
print(f'F1 Score: {f1:.2f}')
print(f'Precision: {precision:.2f}')
print(f'Recall: {recall:.2f}')
print(f'Specificity: {np.mean(specificity):.2f}')

Confusion Matrix:
[[118  14]
 [ 32  88]]
F1 Score: 0.82
Precision: 0.82
Recall: 0.82
Specificity: 0.81
